## Clustering of clients by warehouses + Sequential search, with nested selection of the optimal drone

The solution was written by a team of St. Petersburg State University students.

Developers: Mikhail Posevin, Anastasia Markelova 
###### Academic Advisor: Dr. Ovanes Petrosian http://www.apmath.spbu.ru/en/staff/petrosjanol/index.html

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear\algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import matplotlib.pyplot as plt
import holoviews as hv
import seaborn as sns 
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
from glob import glob

import numpy as np
import pandas as pd

# Data reading

In [ ]:
with open('../input/hashcode-drone-delivery/busy_day.in') as file:
    data_list = file.read().splitlines()

In [ ]:
data_list[2]
print('rows of grid,columns of grid,drones,turns, maxpay load in units(u):',data_list[0], 
      '\n Different product types:',data_list[1],
     '\n product types weigh:',data_list[2],
      '\n warehouses:',data_list[3],
      '\n First warehouse location at first warehouse (row, column):',data_list[4],
      '\n Inventory of products:',data_list[5],
      '\n second warehouse location (row, column)  :',data_list[6],
      '\n Inventory of products at second ware house:',data_list[7],
      '\n Number of orders:',data_list[24],
      '\n First order to be delivery at:',data_list[25],
      '\n Number of items in order:',data_list[26],
      '\n Items of product types:',data_list[27]    )

In [ ]:
def list_lines(lines):
    """Returns a list of integer lists."""
    line_list = [[int(n) for  n in ll.split()] for ll in lines]
    return line_list


def set_params(line_list):
    top_line = line_list[0]
    params = {'DRONE_COUNT': top_line[2],
              'WT_CAP': top_line[4],
              'END_TIME': top_line[3],
              }
    return params


def find_wh_lines(line_list):
    """Provides the dividing line between warehouse and
    order sections in the line list."""
    wh_count = line_list[3][0]
    wh_endline = (wh_count*2)+4
    return wh_endline


def get_weights(line_list):
    weights = np.array(line_list[2])
    return weights.astype(np.int16)


def get_inventories(line_list):
    """Returns a 2-d array of P products by W warehouses."""
    wh_endline = find_wh_lines(line_list)
    invs = line_list[5:wh_endline+1:2]
    supply = np.array(invs).transpose()
    return supply.astype(np.int16)


def get_orders(line_list):
    """Returns a 2-d array of P products by C orders."""
    wh_endline = find_wh_lines(line_list)
    demand = np.zeros((line_list[1][0], line_list[wh_endline][0]),
                            dtype=np.int16)
    orders = line_list[wh_endline+3::3]
    for i,ord in enumerate(orders):
        for prod in ord:
            demand[prod, i] += 1
    return demand.astype(np.int16)


def get_locs(line_list):
    wh_endline = find_wh_lines(line_list)
    wh_locs = np.array(line_list[4:wh_endline:2])
    cust_locs = np.array(line_list[wh_endline+1::3])
    return wh_locs.astype(np.int16), cust_locs.astype(np.int16)
def xysplit(ary):
    return ary[:,:-1].flatten().tolist(), ary[:,-1].flatten().tolist()

In [ ]:
line_list = list_lines(data_list)

params = set_params(line_list)
supply = get_inventories(line_list)
demand = get_orders(line_list)
wh_locs, cust_locs = get_locs(line_list)
weights = get_weights(line_list)

In [ ]:
# The number of different types of products in 10 warehouses + information about the weight of the product
cols=[f'warehouse_{i}' for i in range(1,11)]
Prod_in_warehouses_df = pd.DataFrame(supply.T).T
Prod_in_warehouses_df.columns=cols
Prod_in_warehouses_df['weights'] = weights
Prod_in_warehouses_df['count']  =  [sum(supply[i]) for i in range(400)]
Prod_in_warehouses_df

In [ ]:
#The coordinates of the warehouses
X_warehouses, y_warehouses = xysplit(wh_locs)
warehouse_df = pd.DataFrame({'X_warehouses': X_warehouses, 'y_warehouses': y_warehouses}).astype(np.int32)
warehouse_df

In [ ]:
#The coordinates of the customers
X_customers, y_customers = xysplit(cust_locs)
customers_df = pd.DataFrame({'X_customers': X_customers, 'y_customers': y_customers}).astype(np.uint16)
customers_df

In [ ]:
#Count of each product type in the order
cols_order =[f'prod_{i}' for i in range(1,401)]
Prod_in_order_df = pd.DataFrame(demand.T).fillna(0)
Prod_in_order_df.columns=cols_order
w = []
for i in range(0,1250):
    w.append(sum(np.multiply(Prod_in_order_df.iloc[i:i+1].astype(int).values.tolist()[0],Prod_in_warehouses_df['weights'])))
Prod_in_order_df['total weight'] = w
Prod_in_order_df['count'] = [sum(demand.T[i]) for i in range(1250)]
Prod_in_order_df

In [ ]:
# Full information about the orders
order_information = Prod_in_order_df.join(customers_df)
order = [f'order №{i}' for i in range(1,1251)]
order_information.index = order 
order_information.insert(0, 'id', [i for i in range(1,1251)])
order_information

In [ ]:
# Full information about the warehouses
cols_order =[f'prod_{i}' for i in range(1,401)]
warehouses_information = pd.DataFrame(supply.T)
warehouses_information .columns=cols_order
warehouses_information ['count'] = [sum(supply.T[i]) for i in range(10)]
sum_weights = []
for i in Prod_in_warehouses_df.columns[0:10]:
        sum_weights.append(sum(np.multiply(list(Prod_in_warehouses_df[str(i)]),Prod_in_warehouses_df['weights'])))
warehouses_information ['total weight'] = sum_weights
warehouses_information = warehouses_information.join(warehouse_df)
warehouses_information.index = cols 
warehouses_information.insert(0, 'id', [i for i in range(1,11)])
warehouses_information

# Visualization

In [ ]:
plt.style.use ('ggplot')

In [ ]:
#The weight distribution of the products
fig=plt.figure(figsize=(15,10)) 
ax = sns.distplot(Prod_in_warehouses_df['weights'])

In [ ]:
#Distribution of the number of products in the order
fig=plt.figure(figsize=(15,10)) 
ax = sns.distplot(Prod_in_order_df['count'])

In [ ]:
#Distribution of goods in warehouses
fig=plt.figure(figsize=(15,10)) 
ax = sns.distplot(Prod_in_warehouses_df['count'])

In [ ]:
#The weight distribution of the order
fig=plt.figure(figsize=(15,10)) 
ax = sns.distplot(Prod_in_order_df['total weight'])

In [ ]:
# Location of warehouses and customers
x = range(400)
y = range(400,600)
fig_ = plt.figure(figsize=(12,10))
ax1 = fig_.add_subplot(111)
colors_1 = np.random.rand(50)
ax1.scatter(warehouse_df['X_warehouses'][0], warehouse_df['y_warehouses'][0], s=150,c = 'b', label='Park drones',alpha=1)
ax1.scatter(warehouse_df['X_warehouses'], warehouse_df['y_warehouses'], s=50,c = 'b', label='warehouses',alpha=0.8)
ax1.scatter(customers_df['X_customers'], customers_df['y_customers'], s=20, marker="^", label='orders',alpha=0.5)
plt.legend(loc='upper left')
ax.grid(linewidth = 2,linestyle = '--') 
plt.show()

In [ ]:
fig=plt.figure(figsize=(15,10)) 
ax = sns.boxplot(x ='count', y = 'total weight' ,data = Prod_in_order_df)

In [ ]:
fig=plt.figure(figsize=(15,10)) 
ax = sns.boxplot(Prod_in_order_df['total weight'])

In [ ]:
# Warehouse fill density analysis
fig = px.scatter(warehouses_information, x='X_warehouses', y='y_warehouses', size = 'total weight',color="count",text='id')
iplot(fig)

In [ ]:
#Analytics by weight and quantity of products in the order
fig = px.scatter(order_information, x='X_customers', y='y_customers', size = 'count',color="total weight",text='id')
iplot(fig)

## Clustering of clients by warehouses

In [ ]:
# Clustering by warehouse
def cluster_points(X, mu):
    clusters  = {}
    nearest_wh = []
    for x in X:
        bestmukey = min([(i[0], np.linalg.norm(x-mu[i[0]])) for i in enumerate(mu)], key=lambda t:t[1])[0]
        nearest_wh += [bestmukey]
        try:
            clusters[bestmukey].append(x)
        except KeyError:
            clusters[bestmukey] = [x]
    return clusters, nearest_wh
clsr, nearest_wh = cluster_points(cust_locs , wh_locs)
x = range(400)
y = range(400,600)
fig = plt.figure(figsize=(12,10))
ax1 = fig.add_subplot(111)
colors_1 = np.random.rand(50)
coll = ['b','g','r' ,'c','m','y','k' ,'maroon','indigo','navy']
for i in range(10):
    x, y = zip(*clsr.get(i))
    ax1.scatter(x, y,c = coll[i], s=20,alpha= 0.3)
    ax1.scatter(warehouse_df['X_warehouses'][i], warehouse_df['y_warehouses'][i],c = coll[i],marker="s", s=150,label=f'warehouses{i}',alpha=1)
plt.legend(loc='upper left')
ax1.grid(axis = 'both')
plt.show()

# добавляем информацию о ближайшем складе

order_information['nearest_wh'] = nearest_wh

In [ ]:
# считаем сколько нужно отправить дронов к каждому складу

drons_count = int(data_list[0].split(' ')[2])
wh_count = len(warehouse_df)
orders_count_near_i_wh = []
for i in range(wh_count):
    orders_count_near_i_wh += [len(clsr.get(i))]
orders_count_near_i_wh = np.array(orders_count_near_i_wh)

orders_count_near_i_wh = np.around(orders_count_near_i_wh/(orders_count_near_i_wh.sum()/drons_count))
if sum(orders_count_near_i_wh == 0):
    drons_count_in_wh = (orders_count_near_i_wh == 0) + orders_count_near_i_wh - (orders_count_near_i_wh == max(orders_count_near_i_wh))*sum(orders_count_near_i_wh == 0)
    drons_count_in_wh = np.int0(drons_count_in_wh)

## The distribution of drones

In [ ]:
# распределение дронов

drons_count_in_wh
drons_distribution = []
j = 0
for i in range(len(drons_count_in_wh)):
    drons_distribution += [list(np.array(range(0,drons_count_in_wh[i])) + j)]
    j += drons_count_in_wh[i]

In [ ]:
drons_location = [list(warehouse_df.iloc[0].values)]*30
goods_weight = np.array(data_list[2].split(' '), dtype='int16')

In [ ]:
# считаем взаиморасположение складов

wh_relative_location = np.zeros([wh_count,wh_count])
for i in range(wh_count):
    for j in range(wh_count):
        wh_relative_location[i,j] = np.ceil(np.linalg.norm(warehouse_df.iloc[i] - warehouse_df.iloc[j]))
wh_relative_location = pd.DataFrame(wh_relative_location)

## Search for warehouses with the right products

In [ ]:
# поиск складов с нужными товарами

def availability_check(order_num):
    
    # номер, кол-во и вес товаров в рассматриваемом заказе
    order_goods = order_information.iloc[order_num].values[1:401]
    goods_num = np.nonzero(order_goods)[0]
    order_goods_info = np.array([[i, order_goods[i], goods_weight[i]] for i in goods_num])
    if sum(order_goods_info[:,1] > 1):
        for i in range(len(order_goods_info)):
            if order_goods_info[i][1] != 1:
                k = order_goods_info[i][1]
                order_goods_info[i][1] = 1
                for j in range(k - 1):
                    order_goods_info = np.append(order_goods_info, np.reshape(order_goods_info[i],[1,3]), axis = 0)

    
    tumbler1 = 0
    tumbler2 = 0
    order_nearest_wh = order_information['nearest_wh'][order_num]
    current_wh = order_nearest_wh

    str_with_0_product = []
    while sum(order_goods_info[:,0] == 0):
        warehouses_with_0_product = (warehouses_information['prod_1'][warehouses_information['prod_1'] == 1]).index.tolist()
        warehouses_with_0_product = [int(w.split('_')[-1]) - 1 for w in warehouses_with_0_product]
        for k in wh_relative_location.iloc[order_nearest_wh].sort_values().index.tolist():
            if k in warehouses_with_0_product:
                str_num = order_goods_info[:,0].tolist().index(0)
                str_with_0_product += [order_goods_info[str_num].tolist() + [k]]
                order_goods_info = np.delete(order_goods_info, str_num, axis = 0)
                tumbler2 = 1
                break            
    if current_wh:
        availability = ((warehouses_information.iloc[current_wh].values[1:401])[order_goods_info[:,0].tolist()]!=0) * current_wh
    else:
        availability = ((warehouses_information.iloc[0].values[1:401])[order_goods_info[:,0].tolist()]!=0) * -1
        tumbler1 = 1
    checked_warehouses = 1            

    while sum(availability == 0):
        current_wh = wh_relative_location.iloc[order_nearest_wh].sort_values().index[checked_warehouses]
        necessary_goods = (availability == 0) * order_goods_info[:,0]
        if current_wh:
            availability += (((warehouses_information.iloc[current_wh].values[1:401])[necessary_goods]*(necessary_goods != 0)) != 0 ) * current_wh
        else:
            availability += (((warehouses_information.iloc[0].values[1:401])[necessary_goods]*(necessary_goods != 0)) != 0 ) * -1
            tumbler1 = 1
        checked_warehouses += 1
    order_goods_info = [list(order_goods_info[i]) + [availability[i]] for i in range(len(order_goods_info))]

    if tumbler1:
        for i in range(len(order_goods_info)):
            if order_goods_info[i].count(-1):
                order_goods_info[i][-1] = 0
    if tumbler2:
        for k in range(len(str_with_0_product)):
            order_goods_info = np.concatenate([order_goods_info, np.array(str_with_0_product[k]).reshape([1,4])])            
    availability = pd.DataFrame(order_goods_info, columns = ['num', 'count', 'weight', 'warehouse'])

    indexes = availability.groupby(['num']).count().query('count > 1').index.tolist()
    for index1 in indexes:
        availability1 = availability.query('num == {}'.format(index1))
        count1 = availability.groupby(['num']).count().query('count > 1').loc[index1][0]
        wh1 = availability.query('num == {}'.format(index1)).warehouse.values[0]
        real_count1 = warehouses_information.iat[wh1, index1 + 1]
        k = 1
        while count1 - real_count1 > 0:
            wh2 = wh_relative_location[wh1].tolist().index(sorted(wh_relative_location[wh1].values[warehouses_information['prod_{}'.format(index1 + 1)].values != 0])[k])
            real_count2 = warehouses_information.iat[wh2, index1 + 1]
            if real_count2 >= count1 - real_count1:
                for i in range(real_count1, count1):
                    availability.iat[availability1.index[i],3] = wh2
                    real_count1 = count1
#             else:
#                 for i in range(real_count1, count1 - real_count2):
#                     availability.iat[availability1.index[i],3] = wh2
#                     k += 1
#                     real_count1 += real_count2
    return availability

## Sequential search

In [ ]:
T = int(data_list[0].split(' ')[-2])
t = [0 for i in range(drons_count)]
comand_count = [0 for i in range(drons_count)]
total_score = 0
comands = ''
t_list = [0]*1250
count = 0 
for i in range(len(order_information)):
    availability = availability_check(i)
    score = 0

    cur_ts = t[drons_distribution[order_information.iloc[i][-1]][0]:drons_distribution[order_information.iloc[i][-1]][-1]+1]
    dron_num = drons_distribution[order_information.iloc[i][-1]][cur_ts.index(min(cur_ts))]

    while len(availability) > 0:
        whs = list(set(availability.warehouse.tolist()))
        viewed_whs = []
        item_group = []
        item_group2 = []
        total_sum = 0

        df3 = availability.copy()
        while len(df3) > 0:
            df1 = ((warehouse_df.query('index in {}'.format(whs)) - drons_location[dron_num])**2)
            df1 = df1['X_warehouses'] + df1['y_warehouses']
            viewed_whs += [df1[df1 == df1.min()].index[0]]
            df2 = availability.query('warehouse in {}'.format(viewed_whs))
            df2 = df2.drop(item_group, axis=0)
            
            for j in range(len(df2)):
                if total_sum + df2.iloc[j].weight <= 200:
                    total_sum += df2.iloc[j].weight
                    item_group += [df2.index[j]]
                    item_group2 += [df2.iloc[j].num]
                    wh = df2.iloc[j].warehouse
                    distance = np.ceil(np.linalg.norm(drons_location[dron_num] - warehouse_df.iloc[wh]))
                    drons_location[dron_num] = warehouse_df.iloc[wh].tolist()
                    score += distance + 1
                    warehouses_information.iat[wh, df2.iloc[j].num + 1] -= 1
                    comands += str(dron_num) + ' L ' + str(wh) + ' ' + str(df2.iloc[j].num) + ' ' + '1\n'
                    count+=1
                    
            whs.remove(viewed_whs[-1])
            df3 = availability.query('weight + {} <= 200'.format(total_sum))
            df3 = df3.query('warehouse not in {}'.format(viewed_whs))
            drons_location[dron_num] = warehouse_df.iloc[viewed_whs[-1]].tolist()
            whs = list(set(df3.warehouse.tolist()))

        distance = np.ceil(np.linalg.norm(drons_location[dron_num] - order_information.iloc[i][403:405]))
        drons_location[dron_num] = order_information.iloc[i][403:405].tolist()
        score += distance
        for k in item_group2:
            comands += str(dron_num) + ' D ' + str(i) + ' ' + str(k) + ' ' + '1\n'
            count+=1

        availability = availability.query('index not in {}'.format(item_group))
        
        t[dron_num] += score
        total_score += np.ceil((T - t[dron_num])/T*100)
        t_list[i] = total_score 

In [ ]:
count

## The output file

In [ ]:
f = open('submission.csv','w')
f.write('18736\n')
for index in comands:
    f.write(index)
f.close()